# Import libraries

In [1]:
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

import pandas

from params import lsh_test, hypercube_test, gnn_test, mrng_test

In [2]:
input_path = b'../MNIST/input.dat'
query_path = b'../MNIST/query.dat'

n = 60000

# 1. Optimize LSH

To skip logs, click [here](#visualize-lsh-study-results).

In [3]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000)
                 }
    
    average_time, true_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=10, **param_dict, query_trick=True, N=1)

    return maf.value, average_time.value

In [4]:
%%time
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'])
lsh_study.optimize(objective_lsh, n_trials=50)
print("-----------------------------------------------------")

trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2023-11-24 19:42:55,042] A new study created in memory with name: lsh


Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 19:43:11,605] Trial 0 finished with values: [1.1564258406886092, 0.015325] and parameters: {'k': 6, 'L': 6, 'table_size': 3750, 'window_size': 2404}. 


Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 19:43:17,439] Trial 1 finished with values: [1.0, 0.0796542] and parameters: {'k': 3, 'L': 2, 'table_size': 3750, 'window_size': 3411}. 


...

Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 19:56:44,916] Trial 48 finished with values: [1.4385305820029064, 0.0068092] and parameters: {'k': 7, 'L': 3, 'table_size': 15000, 'window_size': 2250}. 


Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 19:57:00,923] Trial 49 finished with values: [1.0, 0.5556825] and parameters: {'k': 2, 'L': 9, 'table_size': 15000, 'window_size': 4259}. 


-----------------------------------------------------
Trial no. 26
 Values = [1.0, 0.0194746]
 Params = {'k': 6, 'L': 3, 'table_size': 3750, 'window_size': 3146}
Trial no. 6
 Values = [1.1308915778266788, 0.0152777]
 Params = {'k': 2, 'L': 3, 'table_size': 7500, 'window_size': 1120}
Trial no. 38
 Values = [1.1609128232011328, 0.004841799999999999]
 Params = {'k': 6, 'L': 6, 'table_size': 15000, 'window_size': 1667}
Trial no. 11
 Values = [1.4151888797371197, 0.0023997999999999997]
 Params = {'k': 6, 'L': 5, 'table_size': 3750, 'window_size': 1514}
Trial no. 10
 Values = [1.685662653483007, 0.0016545999999999998]
 Params = {'k': 8, 'L': 5, 'table_size': 15000, 'window_size': 1902}
Trial no. 40
 Values = [1.9656307013892271, 0.0012372999999999998]
 Params = {'k': 10, 'L': 10, 'table_size': 7500, 'window_size': 1230}
Trial no. 2
 Values = [2.096701335671093, 0.0009851999999999999]
 Params = {'k': 9, 'L': 5, 'table_size': 15000, 'window_size': 1724}
Trial no. 4
 Values = [1.797693134862315

## Visualize LSH study results

In [5]:
plot_pareto_front(lsh_study, target_names=['maf', 'average_time'])

In [6]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [7]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [8]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [9]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')